# Урок 3. Коллаборативная фильтрация

1) Попытаться ответить на вопросы/выдвинуть гипотезы

Пересматривать видео ТОЛЬКО для того, чтобы понять на какие вопросы надо ответить не считаю целесообразным, так как форматирование видео от gb отсутствует как класс и искать в таких "учебных" видео, что-то конкретное крайне затруднительно. Этот пукнт оставляю без ответа.

2) Доделать прошлые домашния задания
В рамках данного курса или всей жизни? В рамках данного курса задания к урокам 1 и 2 сделаны. Данное задание относится к 3 уроку, поэтому пункт выполнен.

3) Прочитать статьи BM25/MatrixFactorization
Статья на [Википедии](https://en.wikipedia.org/wiki/Okapi_BM25#:~:text=BM25%20is%20a%20bag%2Dof,slightly%20different%20components%20and%20parameters) прочтена, формул очень много, а объяснений мало. Лично мне, чтобы постичь этот математический рубеж, либо надо быть знатоком математики, коим не могу себя назвать ДАжЕ после курсов по матиматеке в gb, либо иметь наивысшую мотивацию в виде понимания как часто, как и где я это буду использовать. Увы ни я сам, ни один из преподавателей, мне, как студенту факультета ИСКУССТВЕННОГО ИНТЕЛЕКТА, это не объяснили. Галочку поставил, статья прочтена.

Статья c [ресурса о простой дата саенс](https://datasciencemadesimpler.wordpress.com/tag/alternating-least-squares/) изучена. Она показалась более подробной, нежели предыдущая, но во многом к ней можно применить комментарии, сказанные выше. Кроме того, обе статьи заслуживают комментарии о том, что не все студенты владеют английским языком а общедоступные средства перевода еще не столь качественно переводят подобные тексты.

4) Поэкспериментировать с ALS (grid-search)

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix
from scipy import stats

# Матричная факторизация
import implicit
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight
import warnings
warnings.filterwarnings('ignore')

In [4]:
def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

In [5]:
def get_recommendations(user, model, sparse_user_item, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

In [6]:
data = pd.read_csv('./sample_data/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)

data['norm_retail_disc'] =  (data['retail_disc']-data['retail_disc'].min())/(data['retail_disc'].max() - data['retail_disc'].min())
data['week_no_52'] =  data['week_no'] % 52
data['week_no_52'] =  (data['week_no_52']-data['week_no_52'].min())/(data['week_no_52'].max() - data['week_no_52'].min())

data['sec_trans_time'] = (60*data['trans_time'].astype(int)/100 + data['trans_time'] % 100)*60
data['sec_trans_time'] =  (data['sec_trans_time']-data['sec_trans_time'].min())/(data['sec_trans_time'].max() - data['week_no_52'].min())

test_size_weeks = 3
data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [7]:
item_features = pd.read_csv('./sample_data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

In [8]:
result_train = data_train.groupby('user_id')['item_id'].unique().reset_index()
result_train.columns=['user_id', 'actual']
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

In [9]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

In [10]:
user_item_matrix_quantity = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', aggfunc='count', 
                                  fill_value=0).astype(float)

user_item_matrix_norm_retail_disc = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='norm_retail_disc', aggfunc='mean', 
                                  fill_value=0).astype(float)

user_item_matrix_week_no_52 = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='week_no_52', aggfunc='mean', 
                                  fill_value=0).astype(float)

user_item_matrix_sec_trans_time = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='sec_trans_time', aggfunc=lambda x : stats.mode(x)[0], 
                                  fill_value=0).astype(float)

In [11]:
userids = user_item_matrix_quantity.index.values
itemids = user_item_matrix_quantity.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [15]:
!export OPENBLAS_NUM_THREADS=1

In [14]:
model = AlternatingLeastSquares(factors=100, #k
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True,
                                use_gpu=False, 
                                num_threads=0)

def CheckWeight(model, user_item_matrix, valid):
    sparse_user_item = csr_matrix(user_item_matrix).tocsr()
    model.fit(csr_matrix(user_item_matrix).T, show_progress=False)

    recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                            user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                            N=5, # кол-во рекомендаций 
                            filter_already_liked_items=False, 
                            filter_items=None, 
                            recalculate_user=True)
    valid['als'] = valid['user_id'].apply(lambda x: get_recommendations(x, model=model, sparse_user_item=sparse_user_item, N=5))
    return valid.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

In [16]:
lst_user_item_matrix = [user_item_matrix_quantity, user_item_matrix_norm_retail_disc, 
user_item_matrix_week_no_52, user_item_matrix_sec_trans_time]

In [17]:
%%time
res = []

for uim in lst_user_item_matrix:
    res.append(CheckWeight(model, uim, result))

CPU times: user 4min 19s, sys: 4min 25s, total: 8min 45s
Wall time: 4min 31s


In [18]:
columns=['quantity', 'retail_disc', 'week_no_52', 'sec_trans_time']
for c,r in zip(columns, res):
    print(c,'\t', r)

quantity 	 0.16229187071498316
retail_disc 	 0.18481880509304363
week_no_52 	 0.18325171400587362
sec_trans_time 	 0.1764936336924558


In [19]:
import collections
import time




BestModel = collections.namedtuple('BestModel', ['model', 'score_valid', 'score_test', 'param','time'])
params = {
   'factors' : [25, 50, 75, 100, 300],
   'regularization' : [1, 0.1, 0.01, 0.05, 0.001]
}
best_model = BestModel(model=None, score_valid=0, score_test=0, param=None, time=0)

In [22]:
%%time
models=[]
for f in params['factors']:
    for r in params['regularization']:
        start_time = time.time()
        model = AlternatingLeastSquares(factors=f, #k
                                        regularization=r,
                                        iterations=15, 
                                        calculate_training_loss=True,
                                        use_gpu=False, 
                                        num_threads=0)
        score_valid = CheckWeight(model,user_item_matrix_norm_retail_disc,result)
        score_test = CheckWeight(model,user_item_matrix_norm_retail_disc,result_train)
        tm = (time.time() - start_time)
        models.append(BestModel(model, score_valid, score_test,
                                {'factors' : f, 'regularization' : r}, tm))
        if best_model.score_valid < score_valid:
            best_model = models[-1]
        print(f'Checked: factors={f} regularization={r}')

Checked: factors=25 regularization=1
Checked: factors=25 regularization=0.1
Checked: factors=25 regularization=0.01
Checked: factors=25 regularization=0.05
Checked: factors=25 regularization=0.001
Checked: factors=50 regularization=1
Checked: factors=50 regularization=0.1
Checked: factors=50 regularization=0.01
Checked: factors=50 regularization=0.05
Checked: factors=50 regularization=0.001
Checked: factors=75 regularization=1
Checked: factors=75 regularization=0.1
Checked: factors=75 regularization=0.01
Checked: factors=75 regularization=0.05
Checked: factors=75 regularization=0.001
Checked: factors=100 regularization=1
Checked: factors=100 regularization=0.1
Checked: factors=100 regularization=0.01
Checked: factors=100 regularization=0.05
Checked: factors=100 regularization=0.001
Checked: factors=300 regularization=1
Checked: factors=300 regularization=0.1
Checked: factors=300 regularization=0.01
Checked: factors=300 regularization=0.05
Checked: factors=300 regularization=0.001
CPU t

In [23]:
best_model.param

{'factors': 50, 'regularization': 0.01}

In [24]:
best_model.score_valid

0.19187071498530586